In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from dataclasses import dataclass, field
from typing import List, Tuple

In [22]:
x, y, h = 10, 20, 30

# space = np.meshgrid(
#     np.linspace(0, x-1, x),
#     np.linspace(0, y-1, y),
#     np.linspace(0, h-1, h),
#     indexing='ij'
# )

# viewing_angle = np.array([0, 1, 0])  # looking forward
# print(np.array(space).shape)

# space = np.arange(10*4).reshape((10, 4))
space = np.array([
    [1, 2, 3, 1],
    [1, 2, 3, 1],
])

In [30]:
@dataclass
class Camera:
    focal_length: float
    resolution: Tuple[int, int]

    position: np.ndarray#=field(default_factory=np.array([0., 0., 0.]))
    angle: np.ndarray#=field(default_factory=np.array([0., 1., 0.]))  # angle in degrees

    # def __post_init__(self):
    #     self.c = self.a + self.b

    def _rotation_angle(self, std):
        cos = np.dot(std, self.angle)  # / length of angle, which is 1 as it is a unit vector
        sin = np.linalg.norm(np.cross(std, self.angle))

        return cos, sin

    '''
        Transform world coordinate to pixel coordinate
            - World to camera coordinate
            - Camera to pixel coordinate
    '''
    def transform(self, 
        space,
        world_position
    ):
        # convention assumption on viewing angles
        translation = np.array(self.position - world_position)  # row vector

        psi, theta, phi = self.angle  # R_x, R_y, R_z
        sin_psi, cos_psi = np.sin(np.deg2rad(psi)), np.cos(np.deg2rad(psi))
        sin_theta, cos_theta = np.sin(np.deg2rad(theta)), np.cos(np.deg2rad(theta))
        sin_phi, cos_phi = np.sin(np.deg2rad(phi)), np.cos(np.deg2rad(phi))

        rotation = np.array([
            [cos_theta*cos_phi, sin_psi*sin_theta*cos_phi - cos_psi*sin_phi, cos_psi*sin_theta*cos_phi + sin_psi*sin_phi],
            [cos_theta*sin_phi, sin_psi*sin_theta*sin_phi + cos_psi*cos_theta, cos_psi*sin_theta*sin_phi - sin_psi*sin_theta],
            [-sin_theta, sin_phi*cos_theta, cos_phi*cos_theta]
        ])
        m = np.zeros((4, 4))
        m[0:3, 0:3] = rotation
        m[0:3, -1] = translation
        m[-1, -1] = 1

        intrinsic = np.array([
            [self.focal_length, 0, 0, 0],
            [0, self.focal_length, 0, 0],
            [0, 0, 1, 0]
        ])

        abc = np.array([
            [1/self.resolution[0], 0, 1/self.resolution[0]/2],
            [0, 1/self.resolution[1], 1/self.resolution[1]/2],
            [0, 0, 1]
        ])

        # image_space = np.matmul(space, m)
        image_space = np.matmul(m, space.T).T
        camera_space = np.matmul(intrinsic, image_space.T).T
        pixel_space = np.matmul(abc, camera_space.T).T

        return pixel_space

camera = Camera(16.16003, (1536, 1536), np.array([0, 2.5, 3]), (17.288, 232.093, 0))
pixel_coordinate = camera.transform(space, np.array([0, 0, 0]))
print(pixel_coordinate[0])

[-0.03454222  0.02199254  1.94586423]


In [ ]:
np.dot(
    np.array([2, 2, 1]),
    np.array([1, 0, 0]),
)

In [ ]:
points = np.ones((100, 3))
a = np.ones((4, 4))
b = np.ones()